In [50]:
import os
import re
import numpy as np
import inflect

path = "/home/sarehalli/Code/PTB/tagged/pos/wsj/"

data = []
for x in os.listdir(path):
    for y in os.listdir(path+x):
        with open(path+x+"/"+y) as in_f:
            for line in in_f.readlines():
                for token in line.split():
                    if "/" not in token:
                        continue
                    try:
                        word, tag = re.split(r'(?<!\\)/', token)
                        data.append((word, tag))
                    except:
                        print(token)

data[:10]

S*/NNP&P/NN
S*/NNP&P/NN
AT*/NNP&T/NN
Chiat\/NNP
Chiat\/NNP
S*/NNP&P/NN


[('Robert', 'NNP'),
 ('D.', 'NNP'),
 ('Paster', 'NNP'),
 (',', ','),
 ('49', 'CD'),
 ('years', 'NNS'),
 ('old', 'JJ'),
 (',', ','),
 ('vice', 'NN'),
 ('president', 'NN')]

In [51]:
pl = inflect.engine()

In [16]:
words = list(set([w for w,_ in data]))
tags = list(set([t for _,ts in data for t in ts.split("|")]))
tag_by_word = {w:{t:0 for t in tags} for w in words}

for word,tag in data:
    tag_by_word[word.lower()] = {x:((y+(1/len(tag.split("|")))) if (x in tag.split("|")) else y) 
                                 for x,y in tag_by_word.get(word, {}).items()}



In [25]:
# verify that we have partial counts for ambiguity

for w,ts in tag_by_word.items():
    for t,c in ts.items():
        if c % 1 != 0:
            print(w,t,c)

than IN 566.5
than RB 2.5
blanket NN 4.5
blanket JJ 0.5
spotting VBG 2.5
spotting NN 0.5
satisfied VBN 10.5
satisfied JJ 5.5
resulting VBG 46.5
resulting JJ 2.5
obliged VBN 4.5
obliged JJ 1.5
borrowing VBG 10.5
borrowing NN 33.5
zillion CD 0.5
zillion NN 0.5
closing VBG 48.5
closing NN 65.5
rid VBN 0.5
rid JJ 1.5
data NNS 4.5
data NN 0.5
delaying VBG 9.5
delaying NN 0.5
canceled VBN 25.5
canceled JJ 1.5
worse RBR 2.5
worse JJR 8.5
diluted VBN 18.5
diluted JJ 6.5
bulk-chemical NN 0.5
bulk-chemical JJ 1.5
ketchup VB 0.5
ketchup IN 0.5
zilch NN 0.5
zilch JJ 0.5
electronic-data NNS 0.5
electronic-data NN 0.5
semimonthly JJ 0.5
semimonthly RB 0.5
matters VBZ 2.5
matters NNS 13.5
put VB 6.5
put VBP 1.5
being VBG 197.5
being JJ 1.5
trading VBG 16.5
trading NN 43.5
mighta MD 0.5
mighta VB 0.5
cutting VBG 15.5
cutting NN 4.5
surrounded VBN 6.5
surrounded JJ 0.5
depositary NN 3.5
depositary JJ 3.5
polishing VBG 0.5
polishing NN 1.5
walking VBG 10.5
walking NN 2.5
waterworks NNS 0.5
waterworks NN

In [31]:
infverb_tags = ["VBP", "VBZ"]

tag_by_infverb = {x:y for x,y in tag_by_word.items() if sum([(y.get(z, 0) > 0) for z in infverb_tags]) > 0}

for word,tag_counts in tag_by_infverb.items():
    small_tag_counts = {}
    for tag, count in tag_counts.items():
        if tag in infverb_tags:
            small_tag_counts[tag] = small_tag_counts.get(tag, 0) + count
        else:
            small_tag_counts["other"] = small_tag_counts.get("other", 0) + count
    tag_by_infverb[word] = small_tag_counts
    

In [46]:
tagdist_by_verb = {w:{t:c/sum(ts.values()) for t,c in ts.items()} for w,ts in tag_by_infverb.items()}
tagdist_by_verb 

{'notice': {'other': 0.8333333333333334,
  'VBZ': 0.0,
  'VBP': 0.16666666666666666},
 'settles': {'other': 0.0, 'VBZ': 1.0, 'VBP': 0.0},
 'reacts': {'other': 0.0, 'VBZ': 1.0, 'VBP': 0.0},
 'enjoys': {'other': 0.0, 'VBZ': 1.0, 'VBP': 0.0},
 'ai': {'other': 0.0, 'VBZ': 0.125, 'VBP': 0.875},
 'meet': {'other': 0.75, 'VBZ': 0.0, 'VBP': 0.25},
 'shakes': {'other': 0.0, 'VBZ': 1.0, 'VBP': 0.0},
 'repairs': {'other': 0.95, 'VBZ': 0.05, 'VBP': 0.0},
 'measures': {'other': 0.8051948051948052,
  'VBZ': 0.19480519480519481,
  'VBP': 0.0},
 'declare': {'other': 0.9, 'VBZ': 0.0, 'VBP': 0.1},
 'considers': {'other': 0.0, 'VBZ': 1.0, 'VBP': 0.0},
 'hooks': {'other': 0.5, 'VBZ': 0.5, 'VBP': 0.0},
 'condemns': {'other': 0.0, 'VBZ': 1.0, 'VBP': 0.0},
 'contributes': {'other': 0.0, 'VBZ': 1.0, 'VBP': 0.0},
 'recites': {'other': 0.0, 'VBZ': 1.0, 'VBP': 0.0},
 'alienates': {'other': 0.0, 'VBZ': 1.0, 'VBP': 0.0},
 'counts': {'other': 0.5, 'VBZ': 0.5, 'VBP': 0.0},
 'crave': {'other': 0.0, 'VBZ': 0.0, 'VBP':

In [64]:
balanced_tdbv = {}

word_pairs = []
for word,ts in tagdist_by_verb.items():
    if ts["VBZ"] > 0:
        try:
            plural = pl.plural(word)
            if plural in tagdist_by_verb:
                balanced_tdbv[word] = ts
                balanced_tdbv[plural] = tagdist_by_verb[plural]
                word_pairs.append((word, plural))
        except: 
            print(word)
            
len(word_pairs)

's


557

In [62]:
import pickle

with open("verb_pairs.dict", "wb") as out_f:
    pickle.dump(word_pairs, out_f)

with open("verb_dist.dict", "wb") as out_f:
    pickle.dump(balanced_tdbv, out_f)

In [60]:
with open("verb_dist.dict", "rb") as in_f:
    tdbv = pickle.load(in_f)
    
tdbv

{'settles': {'other': 0.0, 'VBZ': 1.0, 'VBP': 0.0},
 'settle': {'other': 0.9487179487179487,
  'VBZ': 0.0,
  'VBP': 0.05128205128205128},
 'reacts': {'other': 0.0, 'VBZ': 1.0, 'VBP': 0.0},
 'react': {'other': 0.7142857142857143, 'VBZ': 0.0, 'VBP': 0.2857142857142857},
 'enjoys': {'other': 0.0, 'VBZ': 1.0, 'VBP': 0.0},
 'enjoy': {'other': 0.65, 'VBZ': 0.0, 'VBP': 0.35},
 'repairs': {'other': 0.95, 'VBZ': 0.05, 'VBP': 0.0},
 'repair': {'other': 0.9259259259259259,
  'VBZ': 0.0,
  'VBP': 0.07407407407407407},
 'contributes': {'other': 0.0, 'VBZ': 1.0, 'VBP': 0.0},
 'contribute': {'other': 0.6956521739130435,
  'VBZ': 0.0,
  'VBP': 0.30434782608695654},
 'counts': {'other': 0.5, 'VBZ': 0.5, 'VBP': 0.0},
 'count': {'other': 0.8947368421052632,
  'VBZ': 0.0,
  'VBP': 0.10526315789473684},
 'resurrects': {'other': 0.0, 'VBZ': 1.0, 'VBP': 0.0},
 'resurrect': {'other': 0.8, 'VBZ': 0.0, 'VBP': 0.2},
 'doubts': {'other': 0.5555555555555556,
  'VBZ': 0.4444444444444444,
  'VBP': 0.0},
 'doubt': {'

In [34]:
import csv
# convert to dictwriter format
dw_out = []

for w,ts in tagdist_by_verb.items():
    ts["word"] = w
    dw_out.append(ts)
    
with open("verb_dist.csv", "w") as out_f:
    dw = csv.DictWriter(out_f, fieldnames=dw_out[0].keys())
    dw.writeheader()
    dw.writerows(dw_out)

# Run models using eval_ccgmulti with paradigm allverbs_prod

Then move to the R notebook!